In [1]:
import gc
import numpy as np 
import pandas as pd
import optuna
from sklearn.metrics import log_loss
import warnings

warnings.filterwarnings('ignore')



In [2]:
%%time
h5File = "temp_data/h5/selectkbest_data_with_last.h5";
key_train = 'bestkselect_01/train';
key_valid = 'bestkselect_01/valid';
key_test = 'bestkselect_01/test';

train = pd.read_hdf(h5File, key_train);
valid = pd.read_hdf(h5File, key_valid);
test = pd.read_hdf(h5File, key_test);


CPU times: user 11.6 s, sys: 17.2 s, total: 28.7 s
Wall time: 37.3 s


In [3]:
train

,P_2_min,P_2_max,P_2_avg,D_39_max,D_39_avg,B_1_min,B_1_max,B_1_avg,B_2_min,B_2_max,...,D_132_last,D_132_ldiff,D_133_last,D_136_last,D_139_last,D_140_last,D_141_last,D_143_last,D_145_last,S_2_min_m
customer_ID,,,,,,,,,,,,,,,,,,,,,
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.868580,0.960384,0.933824,0.091505,0.010704,0.001930,0.021655,0.012007,1.000242,1.009672,...,0.0,0.0,0.006210,0.000000,0.007186,0.004234,0.005086,0.005810,0.008533,3
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5,0.861109,0.929122,0.899820,0.567403,0.215205,0.006711,0.109644,0.025654,0.819772,1.008534,...,0.0,0.0,0.002996,0.000000,0.002980,0.007479,0.007870,0.003284,0.008514,3
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1,0.797670,0.904482,0.878454,0.009704,0.004181,0.001472,0.009997,0.004386,0.810796,0.819987,...,0.0,0.0,0.009881,0.000000,0.007383,0.006623,0.000964,0.002202,0.003444,3
000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc,0.567442,0.623392,0.598969,0.268476,0.048862,0.005910,0.279991,0.059876,0.812053,1.009999,...,0.0,0.0,0.001789,0.000000,0.002704,0.006184,0.001899,0.008183,0.002983,3
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed,0.805045,0.940382,0.891679,0.008680,0.004644,0.000776,0.009806,0.005941,0.810670,0.819947,...,0.0,0.0,0.005045,0.000000,0.002974,0.004162,0.005764,0.008154,0.000905,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
e0dc8401f13e88ae5fadf2a6cd22ec8f2162a75964a00492ad406dfd437e716f,0.036622,0.428700,0.305832,0.765157,0.145896,0.053321,0.204184,0.145427,0.051535,0.814995,...,0.0,0.0,0.004369,0.000000,0.000320,0.004045,0.002236,0.004224,0.002413,7
e0dcb868e734d5abe60f6a0d9107989aa0cef45c7da5f18642dee0b455c6c311,0.164876,0.384758,0.312491,0.766661,0.255978,0.451088,1.322943,0.693505,0.008950,0.038503,...,0.0,0.0,0.000537,0.000000,0.005447,0.002212,0.008570,0.009164,0.007635,3
e0dcc67c6a5854ab27ac3b799141c2a5c9831471b93a96f1ee62c3e556133956,0.520425,0.677117,0.582655,1.036184,0.308401,0.111283,0.498702,0.325557,0.050609,0.433234,...,0.0,0.0,0.005766,0.000000,0.000154,0.007504,0.007130,0.006720,0.003733,3


In [4]:
valid

customer_ID
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a    0.0
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5    0.0
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1    0.0
000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc    0.0
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed    0.0
                                                                   ... 
e0dc8401f13e88ae5fadf2a6cd22ec8f2162a75964a00492ad406dfd437e716f    1.0
e0dcb868e734d5abe60f6a0d9107989aa0cef45c7da5f18642dee0b455c6c311    1.0
e0dcc67c6a5854ab27ac3b799141c2a5c9831471b93a96f1ee62c3e556133956    0.0
e0dd131940680fb03f52d808f3c1308abf4b057188fc0540614d2a543ab8a1ea    0.0
e0dd17ec1fae28e858808935b974ce142d53238d657018a1a2f129335e62d388    1.0
Name: target, Length: 458913, dtype: float64

In [5]:
test

,P_2_min,P_2_max,P_2_avg,D_39_max,D_39_avg,B_1_min,B_1_max,B_1_avg,B_2_min,B_2_max,...,D_132_last,D_132_ldiff,D_133_last,D_136_last,D_139_last,D_140_last,D_141_last,D_143_last,D_145_last,S_2_min_m
customer_ID,,,,,,,,,,,,,,,,,,,,,
00000469ba478561f23a92a868bd366de6f6527a684c9a2e78fb826dcac3b9b7,0.568930,0.631315,0.601387,0.238794,0.069758,0.007959,0.021672,0.013925,0.810456,1.009347,...,0.000000,0.000000,0.006273,0.0,0.005912,0.001250,0.006543,0.009160,0.003219,2
00001bf2e77ff879fab36aa4fac689b9ba411dae63ae397d4263dafa1daedef5,0.794469,0.913501,0.862166,0.505795,0.154252,0.014187,0.276014,0.052342,1.000782,1.009245,...,0.000000,0.000000,0.002767,0.0,0.004344,0.000866,0.009120,0.002196,0.007780,4
0000210045da4f81e5f122c6bde5c2a617d03eef67f82c5e400fc98e7bd43ce8,0.673112,0.835114,0.748955,0.679547,0.181814,0.001483,0.039697,0.012762,0.810072,1.009582,...,0.000000,0.000000,0.002045,0.0,1.001246,0.008894,0.896224,1.009391,0.092041,10
00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976cf6e56734528702d694,0.428457,0.514222,0.474728,0.683640,0.470512,0.149511,0.309129,0.284608,0.032696,0.205678,...,0.000000,0.000000,0.009377,0.0,1.008246,0.003753,0.919774,1.008100,0.183020,4
00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9a4693dd914fca22557,0.254478,0.425764,0.324100,0.768016,0.353050,0.357828,0.563603,0.456779,0.020167,0.106739,...,0.000000,0.000000,0.007940,0.0,0.006623,0.001140,0.009527,0.009408,0.000525,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c5d60460dba6dedc41e,0.597667,0.646915,0.631613,0.009711,0.003932,0.001874,0.011309,0.007622,0.810936,0.819394,...,0.000000,0.000000,0.009844,0.0,0.003015,0.006851,0.009307,0.009393,0.005294,4
ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3a4f0ca3de613b0b2ad,0.471303,0.634783,0.547049,0.417259,0.056910,0.015243,0.084163,0.049328,0.082507,1.009882,...,0.000000,0.000000,0.001985,0.0,0.007684,0.003375,0.009780,0.008109,0.009981,10
ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475cb095d2443a68030f1,0.195671,0.316648,0.239070,1.119398,0.232943,0.013811,0.051597,0.031747,0.171140,1.009973,...,0.145253,0.053708,0.274885,0.0,0.002304,0.001640,0.002371,0.008154,0.002120,4


In [6]:
X_train = train
y_train = valid
X_test = test

In [7]:
X_train = train[:1000]
y_train = valid[:1000]
X_test = test[:1000]

In [8]:
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=0, stratify=y_train)
print(X_tr.head())
print(X_val.head())
print(y_tr.head())
print(y_val.head())

                                                     P_2_min   P_2_max  \
customer_ID                                                              
0029d45c7fd5b96eaeaef3dc73d88a9aa0f9bf22c010ac7...  0.800555  0.821940   
0008c2f297e1b00bf567c0d2c25f3e3b356f9a3088d2bf4...  0.290994  0.565513   
00676bbba68dc87ccb2485d6c2ced9dd60be9912974ee3c...  0.937742  0.984307   
006d23fc91d62980c6ba065cc68d9489b73553dc48df77a...  0.974451  1.009723   
008cd7f666bf865907cccfd40db106451354c3c6cd4020b...  0.246159  0.639961   

                                                     P_2_avg  D_39_max  \
customer_ID                                                              
0029d45c7fd5b96eaeaef3dc73d88a9aa0f9bf22c010ac7...  0.810323  0.009857   
0008c2f297e1b00bf567c0d2c25f3e3b356f9a3088d2bf4...  0.494971  0.353181   
00676bbba68dc87ccb2485d6c2ced9dd60be9912974ee3c...  0.964542  0.294949   
006d23fc91d62980c6ba065cc68d9489b73553dc48df77a...  1.000254  0.007407   
008cd7f666bf865907cccfd40db106451354c

In [9]:
from kerastuner.tuners import RandomSearch
import kerastuner as kt
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
        if hp.Choice('batchnorm_and_dropout', ['batch', 'dropout', 'both']) == 'batch':
            model.add(layers.BatchNormalization())
        elif hp.Choice('batchnorm_and_dropout', ['batch', 'dropout', 'both']) == 'dropout':
            model.add(layers.Dropout(0.2))
        else:
            model.add(layers.BatchNormalization())
            model.add(layers.Dropout(0.2))
    model.add(layers.Dense(1, activation="sigmoid"))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss=keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=[keras.metrics.BinaryAccuracy(name='acc')])
    return model


In [10]:
tuner = RandomSearch(
    build_model,
    objective='val_acc',
    max_trials=100,
    overwrite=True)


2022-08-22 11:42:13.148861: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
callbacks=[keras.callbacks.EarlyStopping(monitor='val_acc', mode='max', patience=3)]
tuner.search(X_tr, y_tr, validation_data=(X_val, y_val), callbacks=callbacks, epochs=100)


Trial 100 Complete [00h 00m 03s]
val_acc: 0.8999999761581421

Best val_acc So Far: 0.9900000095367432
Total elapsed time: 00h 12m 13s
INFO:tensorflow:Oracle triggered exit


In [15]:
best_hp = tuner.get_best_hyperparameters()[0]
model = build_model(best_hp)
model.fit(X_val, y_val, epochs=1000)


Epoch 1/1000
10/10 [==============================] - 2s 5ms/step - loss: 0.7721 - acc: 0.5900
Epoch 2/1000
10/10 [==============================] - 0s 7ms/step - loss: 0.5542 - acc: 0.7533
Epoch 3/1000
10/10 [==============================] - 0s 7ms/step - loss: 0.4905 - acc: 0.8100
Epoch 4/1000
10/10 [==============================] - 0s 6ms/step - loss: 0.4220 - acc: 0.8467
Epoch 5/1000
10/10 [==============================] - 0s 7ms/step - loss: 0.3439 - acc: 0.8700
Epoch 6/1000
10/10 [==============================] - 0s 6ms/step - loss: 0.3436 - acc: 0.8667
Epoch 7/1000
10/10 [==============================] - 0s 8ms/step - loss: 0.2501 - acc: 0.9333
Epoch 8/1000
10/10 [==============================] - 0s 7ms/step - loss: 0.2215 - acc: 0.9367
Epoch 9/1000
10/10 [==============================] - 0s 6ms/step - loss: 0.2090 - acc: 0.9533
Epoch 10/1000
10/10 [==============================] - 0s 7ms/step - loss: 0.1729 - acc: 0.9633
Epoch 11/1000
10/10 [============================

10/10 [==============================] - 0s 7ms/step - loss: 0.0190 - acc: 0.9933
Epoch 87/1000
10/10 [==============================] - 0s 6ms/step - loss: 0.0848 - acc: 0.9700
Epoch 88/1000
10/10 [==============================] - 0s 6ms/step - loss: 0.0370 - acc: 0.9833
Epoch 89/1000
10/10 [==============================] - 0s 6ms/step - loss: 0.0204 - acc: 0.9933
Epoch 90/1000
10/10 [==============================] - 0s 6ms/step - loss: 0.0202 - acc: 0.9933
Epoch 91/1000
10/10 [==============================] - 0s 6ms/step - loss: 0.0122 - acc: 1.0000
Epoch 92/1000
10/10 [==============================] - 0s 6ms/step - loss: 0.0854 - acc: 0.9700
Epoch 93/1000
10/10 [==============================] - 0s 7ms/step - loss: 0.0158 - acc: 0.9967
Epoch 94/1000
10/10 [==============================] - 0s 7ms/step - loss: 0.0127 - acc: 0.9967
Epoch 95/1000
10/10 [==============================] - 0s 7ms/step - loss: 0.0257 - acc: 0.9900
Epoch 96/1000
10/10 [==============================] -

10/10 [==============================] - 0s 10ms/step - loss: 0.0161 - acc: 0.9933
Epoch 171/1000
10/10 [==============================] - 0s 7ms/step - loss: 0.0095 - acc: 0.9967
Epoch 172/1000
10/10 [==============================] - 0s 7ms/step - loss: 0.0062 - acc: 1.0000
Epoch 173/1000
10/10 [==============================] - 0s 7ms/step - loss: 0.0662 - acc: 0.9667
Epoch 174/1000
10/10 [==============================] - 0s 7ms/step - loss: 0.0139 - acc: 0.9933
Epoch 175/1000
10/10 [==============================] - 0s 7ms/step - loss: 0.0293 - acc: 0.9900
Epoch 176/1000
10/10 [==============================] - 0s 8ms/step - loss: 0.0111 - acc: 0.9967
Epoch 177/1000
10/10 [==============================] - 0s 9ms/step - loss: 0.0183 - acc: 0.9933
Epoch 178/1000
10/10 [==============================] - 0s 7ms/step - loss: 0.0076 - acc: 1.0000
Epoch 179/1000
10/10 [==============================] - 0s 7ms/step - loss: 0.1323 - acc: 0.9733
Epoch 180/1000
10/10 [======================

10/10 [==============================] - 0s 7ms/step - loss: 0.0330 - acc: 0.9900
Epoch 255/1000
10/10 [==============================] - 0s 8ms/step - loss: 0.0159 - acc: 0.9967
Epoch 256/1000
10/10 [==============================] - 0s 8ms/step - loss: 0.0199 - acc: 0.9967
Epoch 257/1000
10/10 [==============================] - 0s 9ms/step - loss: 0.0098 - acc: 1.0000
Epoch 258/1000
10/10 [==============================] - 0s 7ms/step - loss: 0.0044 - acc: 1.0000
Epoch 259/1000
10/10 [==============================] - 0s 7ms/step - loss: 0.0084 - acc: 0.9967
Epoch 260/1000
10/10 [==============================] - 0s 8ms/step - loss: 0.0176 - acc: 0.9967
Epoch 261/1000
10/10 [==============================] - 0s 7ms/step - loss: 0.0178 - acc: 0.9933
Epoch 262/1000
10/10 [==============================] - 0s 7ms/step - loss: 0.0130 - acc: 0.9967
Epoch 263/1000
10/10 [==============================] - 0s 7ms/step - loss: 0.0016 - acc: 1.0000
Epoch 264/1000
10/10 [=======================

10/10 [==============================] - 0s 8ms/step - loss: 0.0101 - acc: 0.9933
Epoch 339/1000
10/10 [==============================] - 0s 8ms/step - loss: 0.0025 - acc: 1.0000
Epoch 340/1000
10/10 [==============================] - 0s 7ms/step - loss: 0.0021 - acc: 1.0000
Epoch 341/1000
10/10 [==============================] - 0s 8ms/step - loss: 0.0060 - acc: 1.0000
Epoch 342/1000
10/10 [==============================] - 0s 8ms/step - loss: 0.0012 - acc: 1.0000
Epoch 343/1000
10/10 [==============================] - 0s 7ms/step - loss: 0.0015 - acc: 1.0000
Epoch 344/1000
10/10 [==============================] - 0s 7ms/step - loss: 0.0211 - acc: 0.9967
Epoch 345/1000
10/10 [==============================] - 0s 7ms/step - loss: 0.0018 - acc: 1.0000
Epoch 346/1000
10/10 [==============================] - 0s 7ms/step - loss: 0.0011 - acc: 1.0000
Epoch 347/1000
10/10 [==============================] - 0s 7ms/step - loss: 0.0108 - acc: 0.9967
Epoch 348/1000
10/10 [=======================

...

10/10 [==============================] - 0s 10ms/step - loss: 0.0021 - acc: 1.0000
Epoch 672/1000
10/10 [==============================] - 0s 8ms/step - loss: 0.0014 - acc: 1.0000
Epoch 673/1000
10/10 [==============================] - 0s 10ms/step - loss: 3.7507e-04 - acc: 1.0000
Epoch 674/1000
10/10 [==============================] - 0s 9ms/step - loss: 0.0014 - acc: 1.0000
Epoch 675/1000
10/10 [==============================] - 0s 9ms/step - loss: 5.7283e-04 - acc: 1.0000
Epoch 676/1000
10/10 [==============================] - 0s 10ms/step - loss: 4.0444e-04 - acc: 1.0000
Epoch 677/1000
10/10 [==============================] - 0s 9ms/step - loss: 7.7581e-04 - acc: 1.0000
Epoch 678/1000
10/10 [==============================] - 0s 10ms/step - loss: 7.2547e-04 - acc: 1.0000
Epoch 679/1000
10/10 [==============================] - 0s 10ms/step - loss: 2.0474e-04 - acc: 1.0000
Epoch 680/1000
10/10 [==============================] - 0s 10ms/step - loss: 2.3733e-04 - acc: 1.0000
Epoch 681/1

10/10 [==============================] - 0s 11ms/step - loss: 0.0038 - acc: 1.0000
Epoch 754/1000
10/10 [==============================] - 0s 10ms/step - loss: 0.0101 - acc: 0.9967
Epoch 755/1000
10/10 [==============================] - 0s 10ms/step - loss: 0.0043 - acc: 1.0000
Epoch 756/1000
10/10 [==============================] - 0s 8ms/step - loss: 0.0064 - acc: 0.9967
Epoch 757/1000
10/10 [==============================] - 0s 9ms/step - loss: 0.0023 - acc: 1.0000
Epoch 758/1000
10/10 [==============================] - 0s 10ms/step - loss: 0.0014 - acc: 1.0000
Epoch 759/1000
10/10 [==============================] - 0s 10ms/step - loss: 0.0106 - acc: 0.9967
Epoch 760/1000
10/10 [==============================] - 0s 9ms/step - loss: 0.0012 - acc: 1.0000
Epoch 761/1000
10/10 [==============================] - 0s 10ms/step - loss: 0.0046 - acc: 1.0000
Epoch 762/1000
10/10 [==============================] - 0s 9ms/step - loss: 0.0013 - acc: 1.0000
Epoch 763/1000
10/10 [=================

10/10 [==============================] - 0s 7ms/step - loss: 5.3319e-04 - acc: 1.0000
Epoch 836/1000
10/10 [==============================] - 0s 9ms/step - loss: 5.9302e-04 - acc: 1.0000
Epoch 837/1000
10/10 [==============================] - 0s 7ms/step - loss: 3.4289e-04 - acc: 1.0000
Epoch 838/1000
10/10 [==============================] - 0s 9ms/step - loss: 6.2740e-04 - acc: 1.0000
Epoch 839/1000
10/10 [==============================] - 0s 7ms/step - loss: 3.2088e-04 - acc: 1.0000
Epoch 840/1000
10/10 [==============================] - 0s 7ms/step - loss: 3.2055e-04 - acc: 1.0000
Epoch 841/1000
10/10 [==============================] - 0s 8ms/step - loss: 0.0098 - acc: 0.9967
Epoch 842/1000
10/10 [==============================] - 0s 7ms/step - loss: 0.0248 - acc: 0.9900
Epoch 843/1000
10/10 [==============================] - 0s 7ms/step - loss: 0.0231 - acc: 0.9933
Epoch 844/1000
10/10 [==============================] - 0s 7ms/step - loss: 0.0253 - acc: 0.9933
Epoch 845/1000
10/10 

10/10 [==============================] - 0s 7ms/step - loss: 0.0044 - acc: 1.0000
Epoch 918/1000
10/10 [==============================] - 0s 8ms/step - loss: 0.0018 - acc: 1.0000
Epoch 919/1000
10/10 [==============================] - 0s 9ms/step - loss: 0.0015 - acc: 1.0000
Epoch 920/1000
10/10 [==============================] - 0s 9ms/step - loss: 0.0021 - acc: 1.0000
Epoch 921/1000
10/10 [==============================] - 0s 7ms/step - loss: 0.0014 - acc: 1.0000
Epoch 922/1000
10/10 [==============================] - 0s 7ms/step - loss: 0.0015 - acc: 1.0000
Epoch 923/1000
10/10 [==============================] - 0s 8ms/step - loss: 0.0015 - acc: 1.0000
Epoch 924/1000
10/10 [==============================] - 0s 8ms/step - loss: 6.4487e-04 - acc: 1.0000
Epoch 925/1000
10/10 [==============================] - 0s 8ms/step - loss: 5.8985e-04 - acc: 1.0000
Epoch 926/1000
10/10 [==============================] - 0s 13ms/step - loss: 0.0014 - acc: 1.0000
Epoch 927/1000
10/10 [==============

10/10 [==============================] - 0s 8ms/step - loss: 0.0013 - acc: 1.0000


In [16]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 448)               224448    
                                                                 
 batch_normalization_10 (Bat  (None, 448)              1792      
 chNormalization)                                                
                                                                 
 dense_13 (Dense)            (None, 416)               186784    
                                                                 
 batch_normalization_11 (Bat  (None, 416)              1664      
 chNormalization)                                                
                                                                 
 dense_14 (Dense)            (None, 32)                13344     
                                                                 
 batch_normalization_12 (Bat  (None, 32)              

In [19]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 8
units_0: 448
batchnorm_and_dropout: batch
units_1: 416
learning_rate: 0.001
units_2: 32
units_3: 32
units_4: 32
units_5: 32
units_6: 32
units_7: 32
Score: 0.9900000095367432
Trial summary
Hyperparameters:
num_layers: 4
units_0: 384
batchnorm_and_dropout: batch
units_1: 128
learning_rate: 0.01
units_2: 512
units_3: 32
units_4: 32
units_5: 32
units_6: 384
units_7: 256
Score: 0.9833333492279053
Trial summary
Hyperparameters:
num_layers: 5
units_0: 352
batchnorm_and_dropout: batch
units_1: 128
learning_rate: 0.01
units_2: 96
units_3: 128
units_4: 128
units_5: 384
units_6: 160
units_7: 384
units_8: 448
units_9: 384
units_10: 448
units_11: 224
units_12: 384
units_13: 64
units_14: 96
units_15: 256
units_16: 384
units_17: 480
Score: 0.9566666483879089
Trial summary
Hyperparameters:
num_layers: 2
units_0: 128
batchnorm_and_dropout: batch
units_1: 192
learning_rate: 0.0001
units_2: 3

In [ ]:
# model.summary() をテキストに変換する
from io import StringIO

with StringIO() as buf:
    # StringIOに書き込む
    model.summary(print_fn=lambda x: buf.write(x + "\n"))
    
    # StringIOから取得
    model_summary_txt = buf.getvalue()
model_summary_txt

In [ ]:
path = f'./output/optuna/503_param-tune_nn.txt'    
s = model_summary_txt
with open(path, mode='w') as f:
    f.write(s)
